In [1]:
from experiments._1_one_user_learn_neighbours.try_some_users import *
from experiments.utils import *
from experiments.datasets import *

/home/pablo/.virtualenvs/tesiscomp/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/pablo/.virtualenvs/tesiscomp/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Switching to API Credentials #3


In [2]:
uid = 1566310694
s = open_session()
user_ld = s.query(User).get(uid)
# X_train, X_test, y_train, y_test = load_or_create_dataset(uid)
X_train, X_test, y_train, y_test = load_or_create_dataframe(uid)

In [3]:
neighbours = get_neighbourhood(uid)

In [4]:
len(user_ld.timeline)

666

In [5]:
from sklearn.ensemble import RandomForestClassifier
clf_class = RandomForestClassifier

# weights for class balancing
w1 = sum(y_train)/len(y_train)
w0 = 1 - w1
sample_weights = np.array([w0 if x==0 else w1 for x in y_train])

print("Training %s" % clf_class.__name__)
clf = clf_class()     
clf.fit(X_train, y_train, sample_weight=sample_weights)

y_true, y_pred = y_train, clf.predict(X_train)

print("Detailed classification report:\n")
print("Scores on training set.\n")
print(classification_report(y_true, y_pred))

y_true, y_pred = y_test, clf.predict(X_test)
print("Scores on test set.\n")
print(classification_report(y_true, y_pred))

Training RandomForestClassifier
Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00    213499
        1.0       1.00      1.00      1.00       467

avg / total       1.00      1.00      1.00    213966

Scores on test set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     91501
        1.0       0.99      1.00      0.99       199

avg / total       1.00      1.00      1.00     91700



Analicemos las probabilidades para tener una idea de la ¿perplejidad? del modelo

In [6]:
maxprobs = []
probas = clf.predict_proba(X_test)
for r in probas:
    maxprobs.append(max(r))

print sum(maxprobs)

print probas.sum()

91695.2
91700.0


Las probabilidades están practicamente concentradas siempre en la clase predicha.

# Análisis de ambigüedades

Este usuario muestra un recall mucho más bajo que los demás.
Esta pérdida de calidad puede deberse a errores forzosos causados por ambigüedades en el dataset de entrenamiento.
Diremos que hay ambigüedad para un vector de features $v$ si $(v,0)$ y $(v,1)$ ocurren ambos una cantidad significativa de veces.

In [7]:
miss_clf_counts, details = count_doomed_samples(X_train, y_train)

In [8]:
details[:30]

[]

In [9]:
miss_clf_counts

defaultdict(float, {})

Las ambigüedades parecen venir mayormente de retweets a usuarios con las siguientes características:

* La mayoría de sus tweets no son retweeteados por LD.
* pocos seguidores en el vecindario de LD

In [10]:
error_sources = [d for d in details if d[1][1.0] >= 10]

Veamos si realmente estos usuarios tienen tan pocos seguidores en el entorno de LD

In [11]:
unames = [x[0][0] for x in error_sources]
users = {u.username: u for u in neighbours if u.username in unames}

In [12]:
g = load_nx_graph()

In [13]:
s = open_session()

In [14]:
followers = []
nids = [u.id for u in neighbours]
for un in unames: # importance order
    fs = get_followers(users[un], s, g)
    fs = [f for f in fs if f.id in nids]
    followers.append((un, fs))

In [15]:
follower_counts = [(un, len(fs)) for (un, fs) in followers]
follower_counts

[]

Parecen tener más seguidores de lo esperado en el entorno. Veamos si estos seguidores retweetean sus tweets o no.

In [16]:
tl_counts = defaultdict(int)
for t in user_ld.timeline:
    tl_counts[t.author_id] += 1

In [17]:
len(tl_counts)

46

In [18]:
tl_counts[148970624]

0

In [19]:
len(user_ld.timeline)

666

In [20]:
def contar_retweets(user, followers):
    count = 0
    for f in followers:
        rts = [t for t in f.timeline if t.author_id == user.id]
        count += len(rts)
    return count  

In [21]:
rt_counts = {}
for un in unames:
    user = users[un]
    rt_counts[un] = contar_retweets(user, fs)

In [22]:
rt_counts

{}